In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine

In [ ]:
#Kaan's Code Starts Here 

#### Use CSV file resources to compile data points on various epidemics by country into a SQL database. Use the parse_dates parameter to parse columns as datetime. This will ensure we can later use the dates as additional data points. 
#### Data source: Kaggle 

In [2]:
dataset_2 = "Data\sars_2003_complete_dataset_clean.csv"
sars_df = pd.read_csv(dataset_2, parse_dates = ["Date"])
sars_df.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


#### Renaming the columns to ensure consistency when the data is loaded into a database. We found these metrics to be most relevant for this dataset. Please keep column names lower case to avoid issues when transferring into SQL database. 

In [3]:
sars_df.columns = ['date', 'country', 'confirmed_cases', 'deaths', 'recoveries']
sars_df.head()

,date,country,confirmed_cases,deaths,recoveries
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


#### The goal is to add additional virus types into the database eventually. For the project purposes, we're focusing on Sars and Covid19. In order to distinguish the virus type - please add in the virus name. 

In [4]:
sars_df['virus_name'] = "Sars"
sars_df.head()

,date,country,confirmed_cases,deaths,recoveries,virus_name
0,2003-03-17,Germany,1,0,0,Sars
1,2003-03-17,Canada,8,2,0,Sars
2,2003-03-17,Singapore,20,0,0,Sars
3,2003-03-17,"Hong Kong SAR, China",95,1,0,Sars
4,2003-03-17,Switzerland,2,0,0,Sars


#### Because there are multiple date formats in each data, we decided to seperate it into year/month/date to keep it consistent across the board. This way regardless of the data format we get in the future, we can always have each date metric readily available for additional queries. 


In [6]:
sars_df['year'] = pd.DatetimeIndex(sars_df['date']).year
sars_df['month'] = pd.DatetimeIndex(sars_df['date']).month
sars_df['day'] = pd.DatetimeIndex(sars_df['date']).day
sars_df.head()

,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day
0,2003-03-17,Germany,1,0,0,Sars,2003,3,17
1,2003-03-17,Canada,8,2,0,Sars,2003,3,17
2,2003-03-17,Singapore,20,0,0,Sars,2003,3,17
3,2003-03-17,"Hong Kong SAR, China",95,1,0,Sars,2003,3,17
4,2003-03-17,Switzerland,2,0,0,Sars,2003,3,17


#### Because each virus happens in different timeframes, we wanted to find a metric to set a common ground on the data. This creates ranking of dates to identify 1st to last confirmed case. We initially wanted to bin the data into groups such as  'first 30 days' or 'first 7 days' but this would limit the ability to query in the database. 

In [8]:
sars_df['day_no'] = (sars_df['date'] - min(sars_df['date'])).dt.days + 1
sars_df.head()

,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day,day_no
0,2003-03-17,Germany,1,0,0,Sars,2003,3,17,1
1,2003-03-17,Canada,8,2,0,Sars,2003,3,17,1
2,2003-03-17,Singapore,20,0,0,Sars,2003,3,17,1
3,2003-03-17,"Hong Kong SAR, China",95,1,0,Sars,2003,3,17,1
4,2003-03-17,Switzerland,2,0,0,Sars,2003,3,17,1


#### Check for naming conventions on countries and make sure they're consistent before uploading into the database. Here for instance, China is listed multiple times under different names. Then rename each one based on most common usage.

In [9]:
sars_df["country"].unique()

array(['Germany', 'Canada', 'Singapore', 'Hong Kong SAR, China',
       'Switzerland', 'Thailand', 'Viet Nam', 'China', 'Taiwan, China',
       'Slovenia', 'United Kingdom', 'Spain', 'United States', 'Italy',
       'Republic of Ireland', 'France', 'Romania', 'Australia', 'Belgium',
       'Brazil', 'Malaysia', 'Kuwait', 'Japan', 'South Africa',
       'Indonesia', 'Philippines', 'Sweden', 'India', 'Mongolia',
       'Bulgaria', 'Republic of Korea', 'Macao SAR, China', 'Poland',
       'New Zealand', 'Colombia', 'Finland', 'Russian Federation'],
      dtype=object)

In [10]:
sars_df = sars_df.replace(to_replace ="Taiwan, China", 
                 value ="Taiwan") 

sars_df = sars_df.replace(to_replace ="Hong Kong SAR, China", 
                 value ="Hong Kong") 

sars_df = sars_df.replace(to_replace ="Macao SAR, China", 
                 value ="Macao") 

sars_df = sars_df.replace(to_replace ="Russian Federation", 
                 value ="Russia") 

sars_df = sars_df.replace(to_replace ="Viet Nam", 
                 value ="Vietnam") 

sars_df = sars_df.replace(to_replace ="Republic of Ireland", 
                 value ="Ireland") 

sars_df = sars_df.replace(to_replace ="Republic of Korea", 
                 value ="South Korea") 


sars_df.head()

,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day,day_no
0,2003-03-17,Germany,1,0,0,Sars,2003,3,17,1
1,2003-03-17,Canada,8,2,0,Sars,2003,3,17,1
2,2003-03-17,Singapore,20,0,0,Sars,2003,3,17,1
3,2003-03-17,Hong Kong,95,1,0,Sars,2003,3,17,1
4,2003-03-17,Switzerland,2,0,0,Sars,2003,3,17,1


#### Creating connection with the SQL Database (once the tables in there are created). Then transferring tables into there for further analysis by other teams. We decided to have relational database due to the nature of the data. 

In [20]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_homework"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [21]:
engine.table_names()

['covid19_df', 'sars_db']

In [22]:
sars_df.to_sql(name='sars_db', con=engine, if_exists='append', index=False)

In [24]:
pd.read_sql_query('select * from sars_db', con=engine).head()

,id,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day,day_no
0,1,2003-03-17,Germany,1,0,0,Sars,2003,3,17,1
1,2,2003-03-17,Canada,8,2,0,Sars,2003,3,17,1
2,3,2003-03-17,Singapore,20,0,0,Sars,2003,3,17,1
3,4,2003-03-17,Hong Kong,95,1,0,Sars,2003,3,17,1
4,5,2003-03-17,Switzerland,2,0,0,Sars,2003,3,17,1


In [ ]:
#Bernt's Code Start here - adding covid19 data into the database 

In [23]:
# Load in df's
dataset_1 = ("Data/covid_19_data.csv")

nCoV_df = pd.read_csv(dataset_1, parse_dates = ["ObservationDate"])
nCoV_df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [24]:
# pandas drop columns using list of column names
nCoV_df = nCoV_df.drop(['SNo', 'Province/State', 'Last Update'], axis=1)
nCoV_df.head()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,2020-01-22,Mainland China,1.0,0.0,0.0
1,2020-01-22,Mainland China,14.0,0.0,0.0
2,2020-01-22,Mainland China,6.0,0.0,0.0
3,2020-01-22,Mainland China,1.0,0.0,0.0
4,2020-01-22,Mainland China,0.0,0.0,0.0


In [25]:
nCoV_df.columns = ["date", "country", "confirmed_cases", "deaths", "recoveries"]
nCoV_df

,date,country,confirmed_cases,deaths,recoveries
0,2020-01-22,Mainland China,1.0,0.0,0.0
1,2020-01-22,Mainland China,14.0,0.0,0.0
2,2020-01-22,Mainland China,6.0,0.0,0.0
3,2020-01-22,Mainland China,1.0,0.0,0.0
4,2020-01-22,Mainland China,0.0,0.0,0.0
...,...,...,...,...,...
3079,2020-03-02,US,1.0,0.0,0.0
3080,2020-03-02,Australia,0.0,0.0,0.0
3081,2020-03-02,US,0.0,0.0,0.0
3082,2020-03-02,US,0.0,0.0,0.0


In [26]:
nCoV_df['virus_name'] = 'Covid19'
nCoV_df.tail()

,date,country,confirmed_cases,deaths,recoveries,virus_name
3079,2020-03-02,US,1.0,0.0,0.0,Covid19
3080,2020-03-02,Australia,0.0,0.0,0.0,Covid19
3081,2020-03-02,US,0.0,0.0,0.0,Covid19
3082,2020-03-02,US,0.0,0.0,0.0,Covid19
3083,2020-03-02,US,0.0,0.0,0.0,Covid19


In [27]:
nCoV_df['year'] = pd.DatetimeIndex(nCoV_df['date']).year
nCoV_df['month'] = pd.DatetimeIndex(nCoV_df['date']).month
nCoV_df['day'] = pd.DatetimeIndex(nCoV_df['date']).day

nCoV_df

,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day
0,2020-01-22,Mainland China,1.0,0.0,0.0,Covid19,2020,1,22
1,2020-01-22,Mainland China,14.0,0.0,0.0,Covid19,2020,1,22
2,2020-01-22,Mainland China,6.0,0.0,0.0,Covid19,2020,1,22
3,2020-01-22,Mainland China,1.0,0.0,0.0,Covid19,2020,1,22
4,2020-01-22,Mainland China,0.0,0.0,0.0,Covid19,2020,1,22
...,...,...,...,...,...,...,...,...,...
3079,2020-03-02,US,1.0,0.0,0.0,Covid19,2020,3,2
3080,2020-03-02,Australia,0.0,0.0,0.0,Covid19,2020,3,2
3081,2020-03-02,US,0.0,0.0,0.0,Covid19,2020,3,2
3082,2020-03-02,US,0.0,0.0,0.0,Covid19,2020,3,2


In [28]:
# Here we are counting what day the incident occurred - based on the first entry recorded by country.
# We discussed this, as we were very interested in trying to figure out how to bin the data.
# We realized that that was getting into data analysis, and decided to leave more granular data 
# available for our database. That way any future analyst could decide for themselves what they'd 
# like to do with it. 

nCoV_df['day_no'] = (nCoV_df['date'] - min(nCoV_df['date'])).dt.days + 1
#nCoV_df.iloc[2000]
nCoV_df.head()

,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day,day_no
0,2020-01-22,Mainland China,1.0,0.0,0.0,Covid19,2020,1,22,1
1,2020-01-22,Mainland China,14.0,0.0,0.0,Covid19,2020,1,22,1
2,2020-01-22,Mainland China,6.0,0.0,0.0,Covid19,2020,1,22,1
3,2020-01-22,Mainland China,1.0,0.0,0.0,Covid19,2020,1,22,1
4,2020-01-22,Mainland China,0.0,0.0,0.0,Covid19,2020,1,22,1


In [29]:
nCoV_df = nCoV_df.replace(to_replace ="Mainland China", 
                 value ="China") 
nCoV_df = nCoV_df.replace(to_replace ="Macau", 
                 value ="Macao") 
nCoV_df = nCoV_df.replace(to_replace ="UK", 
                 value ="United Kingdom")
nCoV_df = nCoV_df.replace(to_replace = "US", 
                value = "United States")

#nCoV_df.head()

nCoV_df["country"].sort_values(ascending=True).unique()

array([' Azerbaijan', 'Afghanistan', 'Algeria', 'Andorra', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Belarus',
       'Belgium', 'Brazil', 'Cambodia', 'Canada', 'China', 'Colombia',
       'Croatia', 'Czech Republic', 'Denmark', 'Dominican Republic',
       'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Hong Kong', 'Iceland', 'India', 'Indonesia',
       'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast',
       'Japan', 'Kuwait', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg',
       'Macao', 'Malaysia', 'Mexico', 'Monaco', 'Morocco', 'Nepal',
       'Netherlands', 'New Zealand', 'Nigeria', 'North Ireland',
       'North Macedonia', 'Norway', 'Oman', 'Others', 'Pakistan',
       'Philippines', 'Portugal', 'Qatar', 'Romania', 'Russia',
       'San Marino', 'Saudi Arabia', 'Senegal', 'Singapore',
       'South Korea', 'Spain', 'Sri Lanka', 'Sweden', 'Switzerland',
       'Taiwan', 'Thailand', 'United Arab Em

In [ ]:
#Bernt's Code Ends here 
#Now loading Bernt's section into the database 

In [30]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_homework"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [31]:
engine.table_names()

['sars_db', 'covid19_db']

In [33]:
nCoV_df.to_sql(name='covid19_db', con=engine, if_exists='append', index=False)

In [34]:
pd.read_sql_query('select * from sars_db', con=engine).head()

,id,date,country,confirmed_cases,deaths,recoveries,virus_name,year,month,day,day_no
0,1,2003-03-17,Germany,1,0,0,Sars,2003,3,17,1
1,2,2003-03-17,Canada,8,2,0,Sars,2003,3,17,1
2,3,2003-03-17,Singapore,20,0,0,Sars,2003,3,17,1
3,4,2003-03-17,Hong Kong,95,1,0,Sars,2003,3,17,1
4,5,2003-03-17,Switzerland,2,0,0,Sars,2003,3,17,1


In [ ]:
#Data has been successfully loaded into the database. 